In [18]:
# packages are loaded
import xarray as xr
import pint_xarray
import glob
import netCDF4 as nc4
import os
import pandas as pd
from   easymore import Easymore
import numpy as np
from easymore import Utility as esmrut

In [19]:
# inputs
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
attr = xr.open_dataset('/home/shg096/scratch/test/attr/attribute.nc')
attr


<xarray.Dataset>
Dimensions:        (n: 410, s: 12, l: 19, m: 15)
Coordinates:
  * n              (n) int64 0 1 2 3 4 5 6 7 ... 402 403 404 405 406 407 408 409
  * s              (s) object '1 clay' '2 silty clay' ... '12 sand'
  * l              (l) object '1 Temperate or sub-polar needleleaf forest' .....
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables: (12/29)
    COMID          (n) int64 ...
    length         (n) float64 ...
    lengthdir      (n) float64 ...
    sinuosity      (n) float64 ...
    slope          (n) float64 ...
    uparea         (n) float64 ...
    ...             ...
    LULC_majority  (n) object ...
    comb_frac      (n, m) float64 ...
    comb_majority  (n) object ...
    comb_soil      (m) object ...
    comb_LULC      (m) object ...
    mean_elev      (n) int64 ...

In [23]:
attr = xr.open_dataset('/home/shg096/scratch/test/attr/attribute.nc')
forcing = xr.open_dataset('/home/shg096/scratch/test/SUMMA/SUMMA_forcing.nc')

# reorder based on the COMID order in the forcing
attr = esmrut.reorder(attr,
                      np.array(forcing['hru'].values),
                      mapping = {'var_id':'COMID','dim_id':'n'})

# prepare for the SUMMA attr file
attr ['hruId'] = attr ['COMID']
attr ['gruId'] = xr.DataArray(attr ['COMID'].values, dims=('gru'))
attr ['hru2gruId'] = attr ['COMID']
attr ['downHRUindex'] = xr.DataArray(np.zeros(len(attr['n'].values)), dims=('hru'))
attr ['elevation'] = attr['mean_elev']
attr ['HRUarea'] = attr['unitarea']
attr ['tan_slope'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['contourLength'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['slopeTypeIndex'] = xr.DataArray(np.ones(len(attr['n'].values)), dims=('hru'))
attr ['soilTypeIndex'] = xr.DataArray(attr ['soil_majority'].values.astype(int), dims=('hru'))
attr ['vegTypeIndex'] = xr.DataArray(attr ['LULC_majority'].values.astype(int), dims=('hru'))
attr ['mHeight'] = xr.DataArray(np.ones(len(attr['n'].values))*2, dims=('hru'))


# List of variables to keep
variables_to_keep = ['hruId', 'gruId', 'hru2gruId', 'downHRUindex',\
                     'elevation', 'HRUarea', 'tan_slope', 'contourLength',\
                     'slopeTypeIndex', 'vegTypeIndex', 'mHeight', 'longitude',\
                     'latitude', 'soilTypeIndex', 'latitude', 'longitude']
# Drop variables not in the list
attr = attr.drop_vars(set(attr.variables) - set(variables_to_keep))

# rename the n dimension to hru
attr = attr.rename({'n': 'hru'})

attr

<xarray.Dataset>
Dimensions:         (hru: 410, gru: 410)
Dimensions without coordinates: hru, gru
Data variables: (12/14)
    latitude        (hru) float64 ...
    longitude       (hru) float64 ...
    hruId           (hru) int64 71032106 71032073 71031850 ... 71022153 71027942
    gruId           (gru) int64 71032106 71032073 71031850 ... 71022153 71027942
    hru2gruId       (hru) int64 71032106 71032073 71031850 ... 71022153 71027942
    downHRUindex    (hru) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...              ...
    tan_slope       (hru) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    contourLength   (hru) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    slopeTypeIndex  (hru) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    soilTypeIndex   (hru) int64 3 3 8 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3 3
    vegTypeIndex    (hru) int64 16 16 16 1 16 1 16 1 ... 8 17 10 17 15 10 15 17
    mHeight         (hru) float64 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0 2.0

In [34]:
# Define dimensions
hru_size = len(attr['hruId'].values)
midSoil_size = 8
midToto_size = 8
ifcToto_size = 9
scalarv_size = 1

# Create a new xarray dataset
ds = xr.Dataset()

# Add dimensions to the dataset
ds['hru'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-'})
ds['midSoil'] = xr.DataArray(range(midSoil_size), dims=('midSoil'))
ds['midToto'] = xr.DataArray(range(midToto_size), dims=('midToto'))
ds['ifcToto'] = xr.DataArray(range(ifcToto_size), dims=('ifcToto'))
ds['scalarv'] = xr.DataArray(range(scalarv_size), dims=('scalarv'))

# Add variables to the dataset
ds['hruId'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-', 'long_name': 'Index of hydrological response unit (HRU)'})
ds['dt_init'] = xr.DataArray([[3600.0] * hru_size], dims=('scalarv', 'hru'))
ds['nSoil'] = xr.DataArray([[8] * hru_size], dims=('scalarv', 'hru'))
ds['nSnow'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyIce'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyLiq'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowDepth'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSWE'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSfcMeltPond'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarAquiferStorage'] = xr.DataArray([[1.0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowAlbedo'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanairTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['mLayerTemp'] = xr.DataArray([[283.16] * hru_size ] * midToto_size , dims=('midToto', 'hru'))
ds['mLayerVolFracIce'] = xr.DataArray([[0] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerVolFracLiq'] = xr.DataArray([[0.2] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerMatricHead'] = xr.DataArray([[-1] * hru_size] * midToto_size, dims=('midSoil', 'hru'))
ds['iLayerHeight'] = xr.DataArray([[4.0] * hru_size] * ifcToto_size, dims=('ifcToto', 'hru'))
ds['mLayerDepth'] = xr.DataArray([[1.5] * hru_size] * midToto_size, dims=('midToto', 'hru'))


# Save the dataset to a NetCDF file
ds  #.to_netcdf('coldState.nc')

if not os.path.isdir(path_to_save):
    os.makedirs(path_to_save)

if os.path.isfile(path_to_save+'SUMMA_forcing.nc'):
    os.remove(path_to_save+'SUMMA_forcing.nc')

ds.to_netcdf(path_to_save+'SUMMA_forcing.nc')

<xarray.Dataset>
Dimensions:               (hru: 410, midSoil: 8, midToto: 8, ifcToto: 9,
                           scalarv: 1)
Coordinates:
  * hru                   (hru) int64 71032106 71032073 ... 71022153 71027942
  * midSoil               (midSoil) int64 0 1 2 3 4 5 6 7
  * midToto               (midToto) int64 0 1 2 3 4 5 6 7
  * ifcToto               (ifcToto) int64 0 1 2 3 4 5 6 7 8
  * scalarv               (scalarv) int64 0
Data variables: (12/19)
    hruId                 (hru) int64 71032106 71032073 ... 71022153 71027942
    dt_init               (scalarv, hru) float64 3.6e+03 3.6e+03 ... 3.6e+03
    nSoil                 (scalarv, hru) int64 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8
    nSnow                 (scalarv, hru) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    scalarCanopyIce       (scalarv, hru) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    scalarCanopyLiq       (scalarv, hru) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    ...                    ...
    mLayerTemp            (midToto, hru) float64 283.2 283.2 ... 283.2 283.2
    mLayerVolFracIce      (midToto, hru) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    mLayerVolFracLiq      (midToto, hru) float64 0.2 0.2 0.2 0.2 ... 0.2 0.2 0.2
    mLayerMatricHead      (midSoil, hru) int64 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1
    iLayerHeight          (ifcToto, hru) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0
    mLayerDepth           (midToto, hru) float64 1.5 1.5 1.5 1.5 ... 1.5 1.5 1.5

[[283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16],
 [283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.16, 283.